In [1]:
import sys  
sys.path.insert(0, 'tutorial')
from tutorial_setup import *

ALIAS: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/aliases.en.tsv.gz"
ALL: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/all.tsv.gz"
CLAIMS: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/claims.tsv.gz"
DESCRIPTION: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/descriptions.en.tsv.gz"
EXAMPLES_DIR: "/Users/amandeep/Github/kgtk/examples"
GE: "/Users/amandeep/Documents/kypher/temp.wikidata_os_v5/graph-embedding"
ISA: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/derived.isa.tsv.gz"
ITEM: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/claims.wikibase-item.tsv.gz"
LABEL: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/labels.en.tsv.gz"
OUT: "/Users/amandeep/Documents/kypher/wikidata_os_v5"
P279: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/derived.P279.tsv.gz"
P279STAR: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/derived.P279star

### Sort the file with all edges and qualifiers for the subgraph

1. Concatenate the edge and qualifier file

In [2]:
!$kgtk sort -i "$TEMP"/all_and_qualifiers.tsv.gz -o "$OUT"/all_and_qualifiers.sorted.1.tsv.gz

        3.19 real         2.40 user         0.20 sys


2. Remove the data type edges 

In [3]:
!$kgtk filter -i "$OUT"/all_and_qualifiers.sorted.1.tsv.gz -p ';data_type,datatype;' \
--invert -o "$OUT"/all_and_qualifiers.sorted.tsv.gz

        5.27 real         5.16 user         0.09 sys


3. Create a data type for generating triples command

In [4]:
!$kgtk filter -i "$TEMP"/kgtk.properties.tsv -p ';data_type;' \
-o "$TEMP/datatypes.kgtk.properties.tsv.gz"

!$kgtk cat -i "$WIKIDATA"/metadata.property.datatypes.tsv.gz  \
"$TEMP"/datatypes.kgtk.properties.tsv.gz \
-o "$OUT"/all.metadata.property.datatypes.tsv

!gzip -f "$OUT"/all.metadata.property.datatypes.tsv

        0.71 real         0.62 user         0.07 sys
        0.78 real         0.67 user         0.07 sys


4. Execute the Triple Generation notebook

In [4]:
pm.execute_notebook(
    os.environ["USECASE_DIR"] + "/Generate-Triples-And-Load-Blazegraph.ipynb",
    os.environ["TEMP"] + "/Generate-Triples-And-Load-Blazegraph-OUT.ipynb",
    parameters=dict(
        kgtk_path = os.environ["OUT"],
        kgtk_file_name = "all_and_qualifiers.sorted.tsv.gz",
        properties_file_path = os.environ["OUT"] + "/all.metadata.property.datatypes.tsv.gz",
        create_image = False,
        load_triples = False
    )
)
;

Executing:   0%|          | 0/14 [00:00<?, ?cell/s]

''

5. Take a peek at the output triples file

In [5]:
!gzcat "$OUT"/all.ttl.gz | head -n 50

@prefix wikibase: <http://wikiba.se/ontology#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
@prefix wdtn: <http://www.wikidata.org/prop/direct-normalized/> .
@prefix wdno: <http://www.wikidata.org/prop/novalue/> .
@prefix wds: <http://www.wikidata.org/entity/statement/> .
@prefix wdv: <http://www.wikidata.org/value/> .
@prefix wdref: <http://www.wikidata.org/reference/> .
@prefix p: <http://www.wikidata.org/prop/> .
@prefix pr: <http://www.wikidata.org/prop/reference/> .
@prefix prv: <http://www.wikidata.org/prop/reference/value/> .
@prefix prn: <http://www.wikidata.org/prop/reference/value-normalized/> .
@prefix ps: <http://www.wikidata.org/prop/statement/> .
@prefix psv: <http://www.wikidata.org/prop/statement/value/> .
@prefix psn: <http://www.wikidata.org/prop/statement/value-normalized/> .
@prefix pq: <http://www.wikidata.org/prop/qualifier/> .
@prefix pqv: <http://www.wikidata.org/prop/qualifier/value/> .
@prefix pqn: <ht